# MPIをつかった並列プログラム

* MPI(Message Passing Interface)は、 並列処理アプリケーション用メッセージパッシングライブラリです。
* メッセージパッシングとは、メッセージと呼ばれる特定の データ形式の受け渡しとこれらの一元的な管理に基づく、通信手段の一つで す。 


* MPI では実行されるプロセス全てにプロセスIDが割り当てられます。
* プロセスIDは0以上の整数であり、一番最初に実行されるプロセスにはID 0が割り当てられ、
* それ以降実行される順に1, 2, 3, ...と割り当てられます。
* プロセスIDが0であるプロセスは親プロセスと呼ばれます。
* また、プロセスIDが1以上のプロセスを子プロセスと呼びます。 
* 通常親プロセスを実行すると、子プロセスはrsh,ssh等によって親プロセスと同じプログラムがリモートホストで実行されます。
* なので、上の例のように、自分のプロセスID(上の例ではmyid)を取得し、実行するコードを分ける必要があります。 

##  MPI通信関数 - MPI_Send,MPI_Recv

* MPI でのメッセージの送受信の代表として MPI_Send、MPI_Recvがあります。
* これらは、ブロッキング(同期)send/recvを行う関数で、ブロッキングとは、sendの場合は相手
* にメッセージの送信が完了するまで、あるいは通信バッファにメッセージを格納し終わるまで、
* このルーチンからは戻ってこないということで、recvの場合は目的のメッセージの受信が完了するまでルーチンから戻ってこないということです。
* また、ノンブロッキング(非同期 )send/recv(MPI_Isend/MPI_Irecv)というのもありますが、ここでは省略します。

# Hello, World!

In [3]:
import ipyparallel as ipp
rc = ipp.Client(profile='mpi')
rc.ids

[0, 1]

In [4]:
%%px
from mpi4py import MPI
print("rank:{} size:{}".format(MPI.COMM_WORLD.Get_rank(),MPI.COMM_WORLD.Get_size()))

[stdout:0] rank:0 size:1
[stdout:1] rank:0 size:1


In [5]:
%%px
from mpi4py import MPI
import sys

size = MPI.COMM_WORLD.Get_size()
rank = MPI.COMM_WORLD.Get_rank()
name = MPI.Get_processor_name()

sys.stdout.write( "Hello, World! I am process {} of {} on {}(name) ".format(rank, size, name))

[stdout:0] Hello, World! I am process 0 of 1 on 7da4744449ba(name) 
[stdout:1] Hello, World! I am process 0 of 1 on 7da4744449ba(name) 


In [6]:
%%px

from mpi4py import MPI
import time

comm = MPI.COMM_WORLD
rank = comm.rank
size = comm.size
name = MPI.Get_processor_name()

shared = (rank+1)*5

if rank == 0:
    data = shared
    print("Hello world: rank {} of {} running on {}".format (rank, size, name))
    for i in range(1, size):
        rank, size, name = comm.recv(source=i, tag=1)
        print("Hello world: rank {} of {} running on {}".format (rank, size, name))
else:
    comm.send((rank, size, name), dest=0, tag=1)


[stdout:0] Hello world: rank 0 of 1 running on 7da4744449ba
[stdout:1] Hello world: rank 0 of 1 running on 7da4744449ba


# Installing MPI Module

In [5]:
!pip install mpi4py

In [1]:
!sudo apt-get update -y
!sudo apt-get install python-mpi4py -y

Hit http://security.debian.org jessie/updates InRelease
Get:1 http://security.debian.org jessie/updates/main amd64 Packages [418 kB]
Ign http://httpredir.debian.org jessie InRelease
Hit http://httpredir.debian.org jessie-updates InRelease
Hit http://httpredir.debian.org jessie Release.gpg
Hit http://httpredir.debian.org jessie Release
Get:2 http://httpredir.debian.org jessie-updates/main amd64 Packages [17.6 kB]
Get:3 http://httpredir.debian.org jessie/main amd64 Packages [9,064 kB]
Fetched 9,499 kB in 25s (378 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following extra packages will be installed:
  libcr0 libgfortran3 libhwloc-plugins libhwloc5 libibverbs1 libnuma1
  libopenmpi1.6 mpi-default-bin ocl-icd-libopencl1 openmpi-bin openmpi-common
Suggested packages:
  blcr-dkms libhwloc-contrib-plugins opencl-icd gfortran openmpi-checkpoint
  python-numpy
The following NEW packages will be installed